In [ ]:
# default_exp tree

# tree
> Tree

In [ ]:
# export
from pyuid3.tree_node import TreeNode
from pyuid3.value import Value
from pyuid3.att_stats import AttStats

In [ ]:
# export
class Tree:
    def __init__(self, root: TreeNode):
        set_root(root)
        
    def get_root(self) -> TreeNode:
        return self.root
    
    def set_root(self, root: TreeNode) -> None:
        self.root = root
        
    def predict(self, i: Instance) -> AttStats:
        test_node = self.get_root()
        while not test_node.is_leaf():
            att_to_test = test_node.get_att()
            r = i.get_reading_for_attribute(att_to_test)
            most_probable = r.get_most_probable()
            
            new_node = None
            for te in test_node.get_edges():
                if te.get_value().get_name() == most_probable.get_name():
                    new_node = te.getChild()
                    break

            if new_node:
                test_node = new_node
            else:
                break

        return test_node.get_stats()
    
    def error(self, i: Instance) -> bool:
        result = self.predict(i)

        return result.get_most_porbable().get_name() == i.get_readings().get_last().get_most_probable().get_name()

    
    #write to file

    def get_class_attribute(self) -> Attribute:
        temp  = root
        while not temp.is_leaf():
            temp = temp.get_edges().get_first().get_child()

        result = Attribute(temp.get_att(), {})
        for v in temp.get_stats().get_statistics():
            result.add_value(v.get_name())
        
        return result

    def fill_rules(self, rules: list, current_rule: list, root: TreeNode) -> list:
        if not current_rule:
            current_rule = []
        
        att_name = root.get_att()
        if not root.is_leaf():
            for e in root.get_edges():
                new_rule = current_rule.copy()
                new_rule.append(Condition(att_name, e.get_value(), "eq"))
                fill_rules(rules, new_rule, e.get_child()) 
            
        else:
            final_rule = current_rule.copy()
            final_rule.append(Condition(att_name, root.get_stats().get_most_porbable(), "set"))
            rules.append(final_rule)

        return rules
                
    def get_rules(self) -> list:
        return fill_rules([], None, get_root())
                
    def fillAttributes(self, result=None, root=None) -> set:
         if result and root:
            att_name = root.get_att()
            att = Attribute(att_name, {},root.get_type())
            att.set_importance_gain(root.get_infogain())
            if att in result:
                for tmp in result:
                    if tmp == att:
                        att = tmp
                        break

            if not root.is_leaf():
                for  e in root.get_edges():
                    att.add_value(e.get_value().get_name())
                    fill_attributes(result, e.get_child())
        
                result.add(att)
            else:
                for v in root.get_stats().get_statistics():
                    att.add_value(v.get_name())
            
                result.add(att)

            return result
        else:
            
            return fill_attributes({}, root)

    def to_dot(self, parent=None) -> str:
        if parent:
            result = ""
            label = parent.get_att() + "\n"
            if parent.is_leaf():
                # Add classification info to leaves
                for v in parent.get_stats().get_statistics():
                    label += str(v) + "\n"     

            result += parent.hash_code() + "[" +
                    "label=\"" + label +"\"," +
                    "shape=box, " +
                    "color=" + (parent.is_leaf() ? "red":"black")+"]" #?????
                    
            for te in parent.get_edges():
                result += parent.hash_code() + "->" + te.get_child().hash_code() +
                "[label=\"" + te.get_value().get_name() + "\n" +
                "conf=" + round(te.get_value().get_confidence() * 100.0) / 100.0 + "\"]\n"
                result += to_dot(te.get_child()) 

            return result;
    
        else:
            result = "digraph mediationTree{\n" 
            result += toDot(root);
            
            return result+"\n}"
   

    def __str__(self, lvl=None, val=None, node=None):
        if lvl and val and node:
            result = "|"
            res = "-------" * lvl
            result = result + res
            result += "is " + val
            if node.isLeaf():
                result += " then " + node.get_att() + " = " + node.get_stats() + "\n"
            else:
                result += "then if " + node.get_att() + " (play=" + node.get_stats() + ")\n"
            lvl += 1
            for te in node.get_edges():
                result += str(lvl, te.get_value(),te.get_child())  
        else:
            result = "if " + root.get_att() + " (play= "+ root.get_stats() + ")\n";
            for te in root.get_edges():
                result += str(lvl, te.get_value(), te.get_child())
            
        return result
    

    
    class Condition:
        def __init__(self, att_name: str, value: Value, op='eq'):
            self.attName = attName
            self.value = value
            self.op = op